In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [7]:
from textsummarizer.constants import *
from textsummarizer.utils.common_func import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
from urllib import request
import zipfile
import os
from textsummarizer.logging import logger
from textsummarizer.utils.common_func import get_size
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


    

In [26]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_tokens(self, example_batch):

        input_encodings = self.tokenizer(example_batch['dialogue'], truncation=True, max_length=1024)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], truncation=True, max_length=128)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert_data(self):
        data_samsum = load_dataset(self.config.data_path)
        data_samsum_temp = data_samsum.map(self.convert_examples_to_tokens, batched=True)
        data_samsum_temp.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [28]:
try:
    config = ConfigurationManager()
    config_data_transformation = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = config_data_transformation)
    data_transformation.convert_data()
except Exception as e:
    raise e

[2025-05-17 16:32:37,028: INFO:  common_func:  Yaml file: config/config.yaml has been loaded succesfully]
[2025-05-17 16:32:37,036: INFO:  common_func:  Yaml file: params.yaml has been loaded succesfully]
[2025-05-17 16:32:37,052: INFO:  common_func:  created directory at: artifacts]
[2025-05-17 16:32:37,057: INFO:  common_func:  created directory at: artifacts/data_transformation]


Map:   0%|          | 0/1 [00:00<?, ? examples/s]


KeyError: 'dialogue'